# Retreival System

> Aim: create a database containing previous puzzles (with there solutions) and create a retreival system that given a new puzzle finds similar puzzles and returns the (best) solutions for the similar puzzles.

In [31]:
import os

In [36]:
# Load all puzzles
# puzzles are stored in a directory structure [year]/[day].txt
PUZZLES_PATH = '/home/twanh/workspace/thesis/puzzles/auto'


In [42]:
# Setup the postgres database
from pgvector.psycopg import register_vector
import psycopg

conn = psycopg.connect(
    "host=localhost dbname=advent_of_agents user=postgres password=postgres",
    autocommit=True,
)

conn.execute('CREATE EXTENSION IF NOT EXISTS vector')

register_vector(conn)

conn.execute('''
-- Puzzles table
CREATE TABLE puzzles (
  id SERIAL PRIMARY KEY,
  year INT NOT NULL,
  day INT NOT NULL,
  full_description TEXT,
  problem_statement TEXT,
  input_format TEXT,
  output_format TEXT,
  test_cases JSONB,
  constraints TEXT,
  keywords TEXT,
  underlying_concepts TEXT,
  embedding VECTOR(1536),  -- Adjust dimension as needed
  UNIQUE(year, day)  -- Ensure year/day combinations are unique
);

-- Create an index on the vector column for fast nearest-neighbor search
CREATE INDEX idx_embedding ON puzzles
  USING ivfflat (embedding vector_l2_ops) WITH (lists = 100);

-- Solutions table with a foreign key to puzzles
CREATE TABLE solutions (
  id SERIAL PRIMARY KEY,
  puzzle_id INT NOT NULL,
  code TEXT NOT NULL,
  author VARCHAR(255),
  source VARCHAR(255),
  created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
  FOREIGN KEY (puzzle_id) REFERENCES puzzles(id) ON DELETE CASCADE
);

-- Index on puzzle_id for faster joins
CREATE INDEX idx_solutions_puzzle_id ON solutions(puzzle_id);

''')

<psycopg.Cursor [COMMAND_OK] [IDLE] (host=localhost user=postgres database=advent_of_agents) at 0x7adc8d5d53d0>

In [46]:
# Get all the paths for the puzzles
def get_puzzle_paths(puzzles_path):
    puzzle_paths = []
    for year in os.listdir(puzzles_path):
        year_path = os.path.join(puzzles_path, year)
        if os.path.isdir(year_path):
            for day in os.listdir(year_path):
                day_path = os.path.join(year_path, day)
                if os.path.isfile(day_path):
                    if day_path.endswith('.txt'):
                        puzzle_paths.append(day_path)
    return puzzle_paths
puzzle_paths = get_puzzle_paths(PUZZLES_PATH)
print(puzzle_paths)
print(f"Found {len(puzzle_paths)} puzzle files.")

['/home/twanh/workspace/thesis/puzzles/auto/2015/day_2_part_1.txt', '/home/twanh/workspace/thesis/puzzles/auto/2015/day_7_part_1.txt', '/home/twanh/workspace/thesis/puzzles/auto/2015/day_3_part_1.txt', '/home/twanh/workspace/thesis/puzzles/auto/2015/day_11_part_1.txt', '/home/twanh/workspace/thesis/puzzles/auto/2015/day_19_part_1.txt', '/home/twanh/workspace/thesis/puzzles/auto/2015/day_15_part_1.txt', '/home/twanh/workspace/thesis/puzzles/auto/2015/day_14_part_1.txt', '/home/twanh/workspace/thesis/puzzles/auto/2015/day_9_part_1.txt', '/home/twanh/workspace/thesis/puzzles/auto/2015/day_5_part_1.txt', '/home/twanh/workspace/thesis/puzzles/auto/2015/day_16_part_1.txt', '/home/twanh/workspace/thesis/puzzles/auto/2015/day_20_part_1.txt', '/home/twanh/workspace/thesis/puzzles/auto/2015/day_8_part_1.txt', '/home/twanh/workspace/thesis/puzzles/auto/2015/day_1_part_1.txt', '/home/twanh/workspace/thesis/puzzles/auto/2015/day_23_part_1.txt', '/home/twanh/workspace/thesis/puzzles/auto/2015/day_6_

In [ ]:
import re
def process_puzzle(path):
    # Extract year and day based on the path (e.g., [year]/day_[day]_0part_1.txt) using regex

    match = re.match(r'(\d{4})/day_(\d+)_0part_1.txt', path)
    year = int(match.group(1))
    day = int(match.group(2))

    # Read the puzzle file
    with open(path, 'r') as f:
        puzzle_raw = f.read()


    

